In [10]:
#chunk 1
import pandas as pd

df = pd.read_csv("../data/raw/data.csv")
df = df.drop_duplicates()
df = df.dropna(subset=["Customer_ID", "Amount", "Product_Category"])
df = df.fillna({"Income": "Unknown", "Feedback": "No Feedback"})
df['Date'] = pd.to_datetime(df['Date'])


In [11]:
#chunk 2
from sklearn.preprocessing import LabelEncoder

cat_cols = ["Gender", "Income", "Customer_Segment", "Product_Category"]
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))


In [12]:
#chunk 3
from prophet import Prophet

df['YearMonth'] = df['Date'].dt.to_period('M')
sales = df.groupby(['YearMonth', 'Product_Category'])['Amount'].sum().reset_index()
sales['YearMonth'] = sales['YearMonth'].dt.to_timestamp()

cat_sales = sales[sales['Product_Category'] == 2][['YearMonth', 'Amount']].copy()  # choose a popular category
cat_sales.rename(columns={'YearMonth':'ds', 'Amount':'y'}, inplace=True)
model = Prophet()

model.fit(cat_sales)
future = model.make_future_dataframe(periods=3, freq='M')
forecast = model.predict(future)
print(forecast[['ds', 'yhat']].tail())


23:00:42 - cmdstanpy - INFO - Chain [1] start processing
23:00:42 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat
10 2024-01-01  1.493495e+06
11 2024-02-01  1.489739e+06
12 2024-02-29  1.486347e+06
13 2024-03-31  1.482591e+06
14 2024-04-30  1.478956e+06


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [14]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = models[1].predict(X_test)

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


NameError: name 'models' is not defined

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

df = pd.read_csv("../data/raw/data.csv")
df = df.drop_duplicates()
df = df.dropna(subset=["Customer_ID", "Amount", "Product_Category"])
df = df.fillna({"Income": "Unknown", "Feedback": "No Feedback"})
df['Date'] = pd.to_datetime(df['Date'])

cat_cols = ["Gender", "Income", "Customer_Segment", "Product_Category"]
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

X = df[["Age", "Gender", "Income", "Amount", "Total_Purchases"]]
y = df["Customer_Segment"]

num_cols = ["Age", "Amount", "Total_Purchases"]
X[num_cols] = SimpleImputer(strategy="median").fit_transform(X[num_cols])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/var/folders/vf/rbbd31d52x71bpbg3sn62p3r0000gn/T/ipykernel_51123/402887826.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = SimpleImputer(strategy="median").fit_transform(X[num_cols])


Test Accuracy: 0.4665515179698399

Classification Report:
               precision    recall  f1-score   support

           0       0.39      0.39      0.39     18180
           1       0.28      0.28      0.28     12834
           2       0.59      0.60      0.59     29155
           3       0.00      0.00      0.00        43

    accuracy                           0.47     60212
   macro avg       0.32      0.32      0.32     60212
weighted avg       0.47      0.47      0.47     60212

